In [1]:
import os

In [2]:
%pwd

'c:\\Users\\user\\Downloads\\Text_Summarizer-main\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\user\\Downloads\\Text_Summarizer-main'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [10]:
import sys
import os

# Add src directory to sys.path
sys.path.append(os.path.join(os.getcwd(), 'src'))

from textSummariser.constants import *
from textSummariser.utils.common import read_yaml, create_directories


In [37]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        # Load the configuration and parameters from YAML files
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        # Create necessary directories
        create_directories([self.config.artifacts_root])

        # Access the 'data_ingestion' section from the config
        data_ingestion_section = self.config.get('data_ingestion', {})

        # Create the DataIngestionConfig object with the configuration data
        self.data_ingestion_config = DataIngestionConfig(config=data_ingestion_section)

    # Method to get the DataIngestionConfig object
    def get_data_ingestion_config(self):
        return self.data_ingestion_config


In [38]:
import os
import urllib.request as request 
import zipfile
from textSummariser.logging import logger
from textSummariser.utils.common import get_size


In [50]:
import os
import urllib.request
import zipfile
import logging

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

class DataIngestion:
    def __init__(self, config):
        self.config = config

    def download_file(self):
        local_data_file = self.config.local_data_file
        # Ensure the full path for data_ingestion folder exists
        directory_path = os.path.dirname(local_data_file)
        
        # Ensure the full directory path exists (both artifacts and data_ingestion)
        if not os.path.exists(directory_path):
            os.makedirs(directory_path)  # Creates parent directories if they don't exist
            logger.info(f"Created directory: {directory_path}")
        
        if not os.path.exists(local_data_file):
            logger.info(f"Downloading file from {self.config.source_URL} to {local_data_file}")
            
            try:
                filename, headers = urllib.request.urlretrieve(url=self.config.source_URL, filename=local_data_file)
                logger.info(f"Downloaded {filename} with the following headers: \n{headers}")
            except Exception as e:
                logger.error(f"Error downloading file: {e}")
        else:
            logger.info(f"{local_data_file} already exists.")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        
        # Ensure the unzip directory exists
        if not os.path.exists(unzip_path):
            os.makedirs(unzip_path)
            logger.info(f"Created directory: {unzip_path}")
        
        try:
            with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
                zip_ref.extractall(unzip_path)
                logger.info(f"Extracted files to {unzip_path}")
        except Exception as e:
            logger.error(f"Error extracting zip file: {e}")


In [51]:
try:
    # Initialize ConfigurationManager to load config.yaml
    config = ConfigurationManager()

    # Get the data ingestion config
    data_ingestion_config = config.get_data_ingestion_config()

    # Initialize DataIngestion with the config
    data_ingestion = DataIngestion(config=data_ingestion_config)

    # Download the file and extract the contents
    data_ingestion_config.download_file()
    data_ingestion_config.extract_zip_file()

except Exception as e:
    logger.error(f"Error during data ingestion process: {e}")


[2024-12-18 11:19:49,931: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-18 11:19:49,935: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-18 11:19:49,939: INFO: common: created directory at: artifacts]
[2024-12-18 11:20:16,826: INFO: 461438356: Downloaded artifacts/data_ingestion/data.zip with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: F0D6:157CD7:D5325:10C5B1:67625CEA
Accept-Ranges: bytes
Date: Wed, 18 Dec 2024 05:49:56 GMT
Via: 1.1 varnish
X-Served-By: cache-del21740-DEL
X-Cache: HIT
X-Cache-Hits: 0
X-Timer: S1734500997.855487,VS0,VE1
Vary: Authorization,Accept-Encodin